In [ ]:
import tensorflow as tf
class NovoLSTM(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(NovoLSTM, self).__init__(**kwargs)
        self.units = units
        self.state_size = [units, units]

    def build(self, input_shape):
        # Weights for input gate
        self.w_i = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='w_i')
        self.u_i = self.add_weight(shape=(self.units, self.units),
                                   initializer='glorot_uniform',
                                   name='u_i')
        self.b_i = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_i')

        self.w_i_prime = self.add_weight(shape=(input_shape[-1], self.units),
                                         initializer='glorot_uniform',
                                         name='w_i_prime')
        self.b_i_prime = self.add_weight(shape=(self.units,),
                                         initializer='zeros',
                                         name='b_i_prime')
        
        # Weights for forget gate
        self.w_f = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='w_f')
        self.u_f = self.add_weight(shape=(self.units, self.units),
                                   initializer='glorot_uniform',
                                   name='u_f')
        self.b_f = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_f')
        
        # Weights for cell state
        self.w_c = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='w_c')
        self.u_c = self.add_weight(shape=(self.units, self.units),
                                   initializer='glorot_uniform',
                                   name='u_c')
        self.b_c = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_c')
        
        # Weights for output gate
        self.w_o = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='glorot_uniform',
                                   name='w_o')
        self.u_o = self.add_weight(shape=(self.units, self.units),
                                   initializer='glorot_uniform',
                                   name='u_o')
        self.b_o = self.add_weight(shape=(self.units,),
                                   initializer='zeros',
                                   name='b_o')
        
        # Layer Normalization
        self.ln = tf.keras.layers.LayerNormalization(axis=-1)
        
        # Mark the layer as built to avoid further automatic build calls
        self.built = True

    def call(self, inputs, states):
        h_prev, c_prev = states
        
        # Input gate
        i_t = self.ln(
            tf.sigmoid(
                tf.matmul(inputs, self.w_i) + tf.matmul(h_prev, self.u_i) + self.b_i
            ) * 
            tf.nn.softmax(
                tf.matmul(inputs, self.w_i_prime) + self.b_i_prime
            )
        )
        
        # Forget gate
        f_t = tf.sigmoid(
            tf.matmul(inputs, self.w_f) + tf.matmul(h_prev, self.u_f) + self.b_f
        )
        
        # Cell update
        c_tilda = tf.tanh(
            tf.matmul(inputs, self.w_c) + tf.matmul(h_prev, self.u_c) + self.b_c
        )
        c_t = f_t * c_prev + i_t * c_tilda
        
        # Output gate
        o_t = tf.sigmoid(
            tf.matmul(inputs, self.w_o) + tf.matmul(h_prev, self.u_o) + self.b_o
        )
        h_t = o_t * tf.tanh(c_t)

        return h_t, [h_t, c_t]
